<a href="https://colab.research.google.com/github/a-wong99/andreas-football-predictor-project/blob/main/Live_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load pkl file in this new worksheet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root = '/content/drive/MyDrive/Andreas personal project'

In [ ]:
import joblib
loaded_model = joblib.load('{}/football-match-prediction-model.pkl'.format(root))

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [14:26:37] WARNING: /workspace/src/common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


In [ ]:
def result(row):
    if row['GF'] == row['GA']:
        res = 'Draw'
    elif row['GF'] > row['GA'] and row['Team Venue'] == 'Home' or row['GF'] < row['GA'] and row['Team Venue'] == 'Away':
        res = 'Home Win'
    else:
        res = 'Away Win'
    return res

We need the features for the latest batch of predictions. First load the historical data we have


In [ ]:
import pandas as pd
import numpy as np

def myfunction(team):
  df_team = pd.read_csv(team)
  df_team = df_team.drop(columns=['Time', 'Day', 'Gls', 'SoT','SoT%','G/Sh','G/SoT','Dist','FK','PK','PKatt','npxG','npxG/Sh','G-xG','np:G-xG','Match Report'])
  df_team.dropna(inplace = True)
  df_team.drop(index=df_team[df_team['Sh'] == 'Sh'].index, inplace=True)
  df_team['Filename'] = os.path.basename(team)
  df_team[['Team','Extra']] = df_team['Filename'].str.split('_', expand=True)
  df_team = df_team.drop(columns=['Filename', 'Extra'])
  df_team.rename(columns={'Venue':'Team Venue', 'Result': 'Result(For Team)'},inplace=True)
  df_team['Match Result'] = df_team.apply(result, axis=1)
  df_team.insert(4, 'Match Result', df_team.pop('Match Result'))
  df_team.insert(2, 'Team', df_team.pop('Team'))
  df_team['cumulative_avg_xG'] = df_team['xG'].expanding().mean()
  df_team['cumulative_avg_GF'] = df_team['GF'].expanding().mean()
  df_team['cumulative_avg_GA'] = df_team['GA'].expanding().mean()
  df_team['cumulative_avg_shots'] = df_team['Sh'].expanding().mean()
  df_team['avg_xG_last_3'] = df_team['xG'].rolling(3).mean()
  df_team['avg_GF_last_3']= df_team['GF'].rolling(3).mean()
  df_team['avg_GA_last_3']= df_team['GA'].rolling(3).mean()
  df_team['avg_Sh_last_3']= df_team['Sh'].rolling(3).mean()
  df_team.rename(columns={'xG':'match_xG','GF':'match_GF','GA':'match_GA','Sh':'match_Sh'}, inplace=True)
  df_team_last=df_team.tail(1)
  return df_team_last
  #return df_team

In [ ]:
import glob
import os

path = '{}/Team data'.format(root)

all_files = glob.glob(os.path.join(path , "*2023-24.csv"))

team_temp = []
print('Teams loaded:')
for filename in all_files:
    df = myfunction(filename)
    team_temp.append(df)
    print(filename)
df_allteam = pd.concat(team_temp, axis=0, ignore_index=True)
df_allteam

Teams loaded:
/content/drive/MyDrive/Andreas personal project/Team data/Liverpool_2023-24.csv
/content/drive/MyDrive/Andreas personal project/Team data/Crystal Palace_2023-24.csv
/content/drive/MyDrive/Andreas personal project/Team data/Brentford_2023-24.csv
/content/drive/MyDrive/Andreas personal project/Team data/Chelsea_2023-24.csv
/content/drive/MyDrive/Andreas personal project/Team data/Manchester City_2023-24.csv
/content/drive/MyDrive/Andreas personal project/Team data/Sheffield Utd_2023-24.csv
/content/drive/MyDrive/Andreas personal project/Team data/Everton_2023-24.csv
/content/drive/MyDrive/Andreas personal project/Team data/Luton Town_2023-24.csv
/content/drive/MyDrive/Andreas personal project/Team data/Bournemouth_2023-24.csv
/content/drive/MyDrive/Andreas personal project/Team data/Newcastle Utd_2023-24.csv
/content/drive/MyDrive/Andreas personal project/Team data/Burnley_2023-24.csv
/content/drive/MyDrive/Andreas personal project/Team data/Brighton_2023-24.csv
/content/dr

,Date,Round,Team,Team Venue,Result(For Team),Match Result,match_GF,match_GA,Opponent,match_Sh,match_xG,cumulative_avg_xG,cumulative_avg_GF,cumulative_avg_GA,cumulative_avg_shots,avg_xG_last_3,avg_GF_last_3,avg_GA_last_3,avg_Sh_last_3
0,21/10/2023,Matchweek 9,Liverpool,Home,W,Home Win,2,0,Everton,25,2.2,2.122222,2.222222,1.000000,16.777778,1.933333,1.666667,1.333333,16.666667
1,21/10/2023,Matchweek 9,Crystal Palace,Away,L,Home Win,0,4,Newcastle Utd,17,1.2,1.133333,0.777778,1.222222,12.777778,0.966667,0.333333,1.333333,11.000000
2,21/10/2023,Matchweek 9,Brentford,Home,W,Home Win,3,0,Burnley,23,3.1,1.877778,1.555556,1.333333,14.888889,1.866667,1.666667,1.000000,17.333333
3,21/10/2023,Matchweek 9,Chelsea,Home,D,Draw,2,2,Arsenal,10,1.3,1.822222,1.444444,1.000000,13.222222,1.633333,2.666667,1.000000,9.666667
4,21/10/2023,Matchweek 9,Manchester City,Home,W,Home Win,2,1,Brighton,10,0.8,1.744444,2.111111,0.777778,15.444444,0.733333,1.000000,1.333333,12.333333
5,21/10/2023,Matchweek 9,Sheffield Utd,Home,L,Away Win,1,2,Manchester Utd,11,1.4,0.844444,0.777778,2.666667,9.111111,1.033333,0.666667,2.333333,10.666667
6,21/10/2023,Matchweek 9,Everton,Away,L,Home Win,0,2,Liverpool,6,0.1,1.633333,1.000000,1.555556,15.444444,1.800000,1.333333,1.333333,18.000000
7,21/10/2023,Matchweek 9,Luton Town,Away,D,Draw,2,2,Nott'ham Forest,13,1.7,1.244444,0.888889,1.888889,12.555556,1.266667,1.000000,1.666667,14.333333
8,21/10/2023,Matchweek 9,Bournemouth,Home,L,Away Win,1,2,Wolves,7,0.6,1.044444,0.666667,2.222222,11.222222,0.633333,0.333333,3.000000,8.666667
9,21/10/2023,Matchweek 9,Newcastle Utd,Home,W,Home Win,4,0,Crystal Palace,10,2.4,2.188889,2.666667,1.000000,13.888889,2.533333,2.666667,0.666667,13.000000


Function to return conditional values

In [ ]:
def result2(row):
    if row['homeGF'] == row['awayGF']:
        res = 'Draw'
    elif row['homeGF'] > row['awayGF']:
        res = 'Home Win'
    else:
        res = 'Away Win'
    return res

Drop columns that we do not need for fixtures

In [ ]:
df_allteam = df_allteam.drop(columns=['Date','Round','Team Venue', 'Result(For Team)','Match Result','Opponent'])
df_allteam

,Team,match_GF,match_GA,match_Sh,match_xG,cumulative_avg_xG,cumulative_avg_GF,cumulative_avg_GA,cumulative_avg_shots,avg_xG_last_3,avg_GF_last_3,avg_GA_last_3,avg_Sh_last_3
0,Liverpool,2,0,25,2.2,2.122222,2.222222,1.000000,16.777778,1.933333,1.666667,1.333333,16.666667
1,Crystal Palace,0,4,17,1.2,1.133333,0.777778,1.222222,12.777778,0.966667,0.333333,1.333333,11.000000
2,Brentford,3,0,23,3.1,1.877778,1.555556,1.333333,14.888889,1.866667,1.666667,1.000000,17.333333
3,Chelsea,2,2,10,1.3,1.822222,1.444444,1.000000,13.222222,1.633333,2.666667,1.000000,9.666667
4,Manchester City,2,1,10,0.8,1.744444,2.111111,0.777778,15.444444,0.733333,1.000000,1.333333,12.333333
5,Sheffield Utd,1,2,11,1.4,0.844444,0.777778,2.666667,9.111111,1.033333,0.666667,2.333333,10.666667
6,Everton,0,2,6,0.1,1.633333,1.000000,1.555556,15.444444,1.800000,1.333333,1.333333,18.000000
7,Luton Town,2,2,13,1.7,1.244444,0.888889,1.888889,12.555556,1.266667,1.000000,1.666667,14.333333
8,Bournemouth,1,2,7,0.6,1.044444,0.666667,2.222222,11.222222,0.633333,0.333333,3.000000,8.666667
9,Newcastle Utd,4,0,10,2.4,2.188889,2.666667,1.000000,13.888889,2.533333,2.666667,0.666667,13.000000


Next, we want to load the next gameweek's matches (Date, Home Team, Away Team)

In [ ]:
df_gwfix = pd.read_csv('{}/2023 24 Fixtures/Gameweek 10 fixtures.csv'.format(root))
df_gwfix = df_gwfix.drop(columns=['Wk', 'Day','Time', 'Attendance','Venue','Referee','Match Report','Notes'])
df_gwfix.rename(columns={'Home' : 'Home Team', 'Away' : 'Away Team'}, inplace=True)
df_gwfix

,Date,Home Team,Away Team
0,27/10/2023,Crystal Palace,Tottenham
1,28/10/2023,Chelsea,Brentford
2,28/10/2023,Bournemouth,Burnley
3,28/10/2023,Arsenal,Sheffield Utd
4,28/10/2023,Wolves,Newcastle Utd
5,29/10/2023,West Ham,Everton
6,29/10/2023,Liverpool,Nott'ham Forest
7,29/10/2023,Brighton,Fulham
8,29/10/2023,Aston Villa,Luton Town
9,29/10/2023,Manchester Utd,Manchester City


Split fixture df into home and away (To make the merging easier)

In [ ]:
df_gwfixhome = df_gwfix.drop(columns=['Away Team'])

We want to replace the team names with their encoded value in order to have the data in a form where we can make a prediction

In [ ]:
df_teamsenc = pd.read_csv('{}/Encode tables/teamencoding.csv'.format(root))
df_teamsenc

,Team,Team encoded
0,Arsenal,0
1,Aston Villa,1
2,Bournemouth,2
3,Brentford,3
4,Brighton,4
5,Burnley,5
6,Chelsea,6
7,Crystal Palace,7
8,Everton,8
9,Fulham,9


In [ ]:
#df_teamsenc.loc['23'] = ['Luton Town', 23]
#df_teamsenc.loc['24'] = ['Sheffield Utd', 24]
#df_teamsenc

,Team,Team encoded
0,Arsenal,0
1,Aston Villa,1
2,Bournemouth,2
3,Brentford,3
4,Brighton,4
5,Burnley,5
6,Chelsea,6
7,Crystal Palace,7
8,Everton,8
9,Fulham,9


In [ ]:
df_gwfixhomemerged = pd.merge(df_gwfixhome, df_allteam, left_on=  ['Home Team'],
                   right_on= ['Team'], how='left')
df_gwfixhomemerged = df_gwfixhomemerged.drop(columns='Team')
df_gwfixhomemerged = pd.merge(df_teamsenc, df_gwfixhomemerged, left_on=['Team'],right_on=['Home Team'], how='right')
df_gwfixhomemerged = df_gwfixhomemerged.drop(columns='Team')
df_gwfixhomemerged.rename(columns={'Team encoded':'Home Team encoded', 'match_GF':'home_team_match_GF', 'match_GA':'home_team_match_GA', 'match_Sh':'home_team_match_Sh', 'match_xG':'home_team_match_xG','cumulative_avg_xG': 'home_team_cumulative_avg_xG', 'cumulative_avg_GF' : 'home_team_cumulative_avg_GF', 'cumulative_avg_GA':'home_team_cumulative_avg_GA', 'cumulative_avg_shots': 'home_team_cumulative_avg_shots','avg_xG_last_3':'home_team_avg_xG_last_3', 'avg_GF_last_3':'home_team_avg_GF_last_3','avg_GA_last_3':'home_team_avg_GA_last_3', 'avg_Sh_last_3':'home_team_avg_Sh_last_3'}, inplace=True)
df_gwfixhomemerged

,Home Team encoded,Date,Home Team,home_team_match_GF,home_team_match_GA,home_team_match_Sh,home_team_match_xG,home_team_cumulative_avg_xG,home_team_cumulative_avg_GF,home_team_cumulative_avg_GA,home_team_cumulative_avg_shots,home_team_avg_xG_last_3,home_team_avg_GF_last_3,home_team_avg_GA_last_3,home_team_avg_Sh_last_3
0,7,27/10/2023,Crystal Palace,0,4,17,1.2,1.133333,0.777778,1.222222,12.777778,0.966667,0.333333,1.333333,11.000000
1,6,28/10/2023,Chelsea,2,2,10,1.3,1.822222,1.444444,1.000000,13.222222,1.633333,2.666667,1.000000,9.666667
2,2,28/10/2023,Bournemouth,1,2,7,0.6,1.044444,0.666667,2.222222,11.222222,0.633333,0.333333,3.000000,8.666667
3,0,28/10/2023,Arsenal,2,2,13,1.0,1.766667,2.000000,0.888889,14.000000,1.600000,2.333333,0.666667,12.666667
4,24,28/10/2023,Wolves,2,1,21,2.4,1.366667,1.222222,1.666667,12.000000,1.500000,1.666667,1.000000,10.666667
5,23,29/10/2023,West Ham,1,4,14,1.1,1.488889,1.777778,1.777778,11.555556,1.466667,1.666667,2.000000,13.000000
6,12,29/10/2023,Liverpool,2,0,25,2.2,2.122222,2.222222,1.000000,16.777778,1.933333,1.666667,1.333333,16.666667
7,4,29/10/2023,Brighton,1,2,5,0.8,1.933333,2.444444,2.000000,15.000000,1.600000,1.333333,3.333333,10.000000
8,1,29/10/2023,Aston Villa,4,1,14,1.9,1.888889,2.555556,1.444444,14.888889,1.700000,3.666667,1.000000,17.000000
9,15,29/10/2023,Manchester Utd,2,1,14,1.1,1.533333,1.222222,1.444444,15.888889,1.266667,1.333333,1.000000,18.000000


In [ ]:
df_gwfixawaymerged = pd.merge(df_gwfix, df_allteam, left_on=  ['Away Team'],
                   right_on= ['Team'], how='left')
df_gwfixawaymerged = df_gwfixawaymerged.drop(columns=['Team'])
df_gwfixawaymerged = pd.merge(df_teamsenc, df_gwfixawaymerged, left_on=['Team'],right_on=['Away Team'], how='right')
df_gwfixawaymerged = df_gwfixawaymerged.drop(columns='Team')
df_gwfixawaymerged.rename(columns={'Team encoded':'Away Team encoded','match_GF':'away_team_match_GF', 'match_GA':'away_team_match_GA', 'match_Sh':'away_team_match_Sh', 'match_xG':'away_team_match_xG','cumulative_avg_xG': 'away_team_cumulative_avg_xG', 'cumulative_avg_GF' : 'away_team_cumulative_avg_GF', 'cumulative_avg_GA':'away_team_cumulative_avg_GA', 'cumulative_avg_shots': 'away_team_cumulative_avg_shots','avg_xG_last_3':'away_team_avg_xG_last_3', 'avg_GF_last_3':'away_team_avg_GF_last_3','avg_GA_last_3':'away_team_avg_GA_last_3', 'avg_Sh_last_3':'away_team_avg_Sh_last_3'}, inplace=True)
df_gwfixawaymerged

,Away Team encoded,Date,Home Team,Away Team,away_team_match_GF,away_team_match_GA,away_team_match_Sh,away_team_match_xG,away_team_cumulative_avg_xG,away_team_cumulative_avg_GF,away_team_cumulative_avg_GA,away_team_cumulative_avg_shots,away_team_avg_xG_last_3,away_team_avg_GF_last_3,away_team_avg_GA_last_3,away_team_avg_Sh_last_3
0,21,27/10/2023,Crystal Palace,Tottenham,2,0,15,1.5,1.822222,2.222222,0.888889,18.666667,1.866667,1.666667,0.333333,18.000000
1,3,28/10/2023,Chelsea,Brentford,3,0,23,3.1,1.877778,1.555556,1.333333,14.888889,1.866667,1.666667,1.000000,17.333333
2,5,28/10/2023,Bournemouth,Burnley,0,3,6,1.1,0.811111,0.777778,2.555556,10.111111,0.933333,1.000000,2.666667,10.000000
3,19,28/10/2023,Arsenal,Sheffield Utd,1,2,11,1.4,0.844444,0.777778,2.666667,9.111111,1.033333,0.666667,2.333333,10.666667
4,16,28/10/2023,Wolves,Newcastle Utd,4,0,10,2.4,2.188889,2.666667,1.000000,13.888889,2.533333,2.666667,0.666667,13.000000
5,8,29/10/2023,West Ham,Everton,0,2,6,0.1,1.633333,1.000000,1.555556,15.444444,1.800000,1.333333,1.333333,18.000000
6,18,29/10/2023,Liverpool,Nott'ham Forest,2,2,19,2.9,1.188889,1.111111,1.333333,11.444444,1.466667,1.000000,1.000000,13.666667
7,9,29/10/2023,Brighton,Fulham,0,2,10,1.0,1.100000,0.888889,1.666667,10.666667,1.433333,1.000000,1.666667,13.333333
8,13,29/10/2023,Aston Villa,Luton Town,2,2,13,1.7,1.244444,0.888889,1.888889,12.555556,1.266667,1.000000,1.666667,14.333333
9,14,29/10/2023,Manchester Utd,Manchester City,2,1,10,0.8,1.744444,2.111111,0.777778,15.444444,0.733333,1.000000,1.333333,12.333333


In [ ]:
df_gwfixallmerged = pd.merge(df_gwfixhomemerged,df_gwfixawaymerged, on=['Date','Home Team'])
df_gwfixallmerged

,Home Team encoded,Date,Home Team,home_team_match_GF,home_team_match_GA,home_team_match_Sh,home_team_match_xG,home_team_cumulative_avg_xG,home_team_cumulative_avg_GF,home_team_cumulative_avg_GA,...,away_team_match_Sh,away_team_match_xG,away_team_cumulative_avg_xG,away_team_cumulative_avg_GF,away_team_cumulative_avg_GA,away_team_cumulative_avg_shots,away_team_avg_xG_last_3,away_team_avg_GF_last_3,away_team_avg_GA_last_3,away_team_avg_Sh_last_3
0,7,27/10/2023,Crystal Palace,0,4,17,1.2,1.133333,0.777778,1.222222,...,15,1.5,1.822222,2.222222,0.888889,18.666667,1.866667,1.666667,0.333333,18.000000
1,6,28/10/2023,Chelsea,2,2,10,1.3,1.822222,1.444444,1.000000,...,23,3.1,1.877778,1.555556,1.333333,14.888889,1.866667,1.666667,1.000000,17.333333
2,2,28/10/2023,Bournemouth,1,2,7,0.6,1.044444,0.666667,2.222222,...,6,1.1,0.811111,0.777778,2.555556,10.111111,0.933333,1.000000,2.666667,10.000000
3,0,28/10/2023,Arsenal,2,2,13,1.0,1.766667,2.000000,0.888889,...,11,1.4,0.844444,0.777778,2.666667,9.111111,1.033333,0.666667,2.333333,10.666667
4,24,28/10/2023,Wolves,2,1,21,2.4,1.366667,1.222222,1.666667,...,10,2.4,2.188889,2.666667,1.000000,13.888889,2.533333,2.666667,0.666667,13.000000
5,23,29/10/2023,West Ham,1,4,14,1.1,1.488889,1.777778,1.777778,...,6,0.1,1.633333,1.000000,1.555556,15.444444,1.800000,1.333333,1.333333,18.000000
6,12,29/10/2023,Liverpool,2,0,25,2.2,2.122222,2.222222,1.000000,...,19,2.9,1.188889,1.111111,1.333333,11.444444,1.466667,1.000000,1.000000,13.666667
7,4,29/10/2023,Brighton,1,2,5,0.8,1.933333,2.444444,2.000000,...,10,1.0,1.100000,0.888889,1.666667,10.666667,1.433333,1.000000,1.666667,13.333333
8,1,29/10/2023,Aston Villa,4,1,14,1.9,1.888889,2.555556,1.444444,...,13,1.7,1.244444,0.888889,1.888889,12.555556,1.266667,1.000000,1.666667,14.333333
9,15,29/10/2023,Manchester Utd,2,1,14,1.1,1.533333,1.222222,1.444444,...,10,0.8,1.744444,2.111111,0.777778,15.444444,0.733333,1.000000,1.333333,12.333333


In [ ]:
df_gwfixallmerged['home_team_avg_xG_last_3'] = df_gwfixallmerged['home_team_avg_xG_last_3'].fillna(df_gwfixallmerged['home_team_cumulative_avg_xG'])
df_gwfixallmerged['home_team_avg_GF_last_3'] = df_gwfixallmerged['home_team_avg_GF_last_3'].fillna(df_gwfixallmerged['home_team_cumulative_avg_GF'])
df_gwfixallmerged['home_team_avg_GA_last_3'] = df_gwfixallmerged['home_team_avg_GA_last_3'].fillna(df_gwfixallmerged['home_team_cumulative_avg_GA'])
df_gwfixallmerged['home_team_avg_Sh_last_3'] = df_gwfixallmerged['home_team_avg_Sh_last_3'].fillna(df_gwfixallmerged['home_team_cumulative_avg_shots'])
df_gwfixallmerged['away_team_avg_xG_last_3'] = df_gwfixallmerged['away_team_avg_xG_last_3'].fillna(df_gwfixallmerged['away_team_cumulative_avg_xG'])
df_gwfixallmerged['away_team_avg_GF_last_3'] = df_gwfixallmerged['away_team_avg_GF_last_3'].fillna(df_gwfixallmerged['away_team_cumulative_avg_GF'])
df_gwfixallmerged['away_team_avg_GA_last_3'] = df_gwfixallmerged['away_team_avg_GA_last_3'].fillna(df_gwfixallmerged['away_team_cumulative_avg_GA'])
df_gwfixallmerged['away_team_avg_Sh_last_3'] = df_gwfixallmerged['away_team_avg_Sh_last_3'].fillna(df_gwfixallmerged['away_team_cumulative_avg_shots'])
df_gwfixallmerged

,Home Team encoded,Date,Home Team,home_team_match_GF,home_team_match_GA,home_team_match_Sh,home_team_match_xG,home_team_cumulative_avg_xG,home_team_cumulative_avg_GF,home_team_cumulative_avg_GA,...,away_team_match_Sh,away_team_match_xG,away_team_cumulative_avg_xG,away_team_cumulative_avg_GF,away_team_cumulative_avg_GA,away_team_cumulative_avg_shots,away_team_avg_xG_last_3,away_team_avg_GF_last_3,away_team_avg_GA_last_3,away_team_avg_Sh_last_3
0,7,27/10/2023,Crystal Palace,0,4,17,1.2,1.133333,0.777778,1.222222,...,15,1.5,1.822222,2.222222,0.888889,18.666667,1.866667,1.666667,0.333333,18.000000
1,6,28/10/2023,Chelsea,2,2,10,1.3,1.822222,1.444444,1.000000,...,23,3.1,1.877778,1.555556,1.333333,14.888889,1.866667,1.666667,1.000000,17.333333
2,2,28/10/2023,Bournemouth,1,2,7,0.6,1.044444,0.666667,2.222222,...,6,1.1,0.811111,0.777778,2.555556,10.111111,0.933333,1.000000,2.666667,10.000000
3,0,28/10/2023,Arsenal,2,2,13,1.0,1.766667,2.000000,0.888889,...,11,1.4,0.844444,0.777778,2.666667,9.111111,1.033333,0.666667,2.333333,10.666667
4,24,28/10/2023,Wolves,2,1,21,2.4,1.366667,1.222222,1.666667,...,10,2.4,2.188889,2.666667,1.000000,13.888889,2.533333,2.666667,0.666667,13.000000
5,23,29/10/2023,West Ham,1,4,14,1.1,1.488889,1.777778,1.777778,...,6,0.1,1.633333,1.000000,1.555556,15.444444,1.800000,1.333333,1.333333,18.000000
6,12,29/10/2023,Liverpool,2,0,25,2.2,2.122222,2.222222,1.000000,...,19,2.9,1.188889,1.111111,1.333333,11.444444,1.466667,1.000000,1.000000,13.666667
7,4,29/10/2023,Brighton,1,2,5,0.8,1.933333,2.444444,2.000000,...,10,1.0,1.100000,0.888889,1.666667,10.666667,1.433333,1.000000,1.666667,13.333333
8,1,29/10/2023,Aston Villa,4,1,14,1.9,1.888889,2.555556,1.444444,...,13,1.7,1.244444,0.888889,1.888889,12.555556,1.266667,1.000000,1.666667,14.333333
9,15,29/10/2023,Manchester Utd,2,1,14,1.1,1.533333,1.222222,1.444444,...,10,0.8,1.744444,2.111111,0.777778,15.444444,0.733333,1.000000,1.333333,12.333333


In [ ]:
df_gwfixfeatures = df_gwfixallmerged.drop(columns=['Date','Home Team','Away Team'])
df_gwfixfeatures.dropna(inplace=True)
df_gwfixfeatures

,Home Team encoded,home_team_match_GF,home_team_match_GA,home_team_match_Sh,home_team_match_xG,home_team_cumulative_avg_xG,home_team_cumulative_avg_GF,home_team_cumulative_avg_GA,home_team_cumulative_avg_shots,home_team_avg_xG_last_3,...,away_team_match_Sh,away_team_match_xG,away_team_cumulative_avg_xG,away_team_cumulative_avg_GF,away_team_cumulative_avg_GA,away_team_cumulative_avg_shots,away_team_avg_xG_last_3,away_team_avg_GF_last_3,away_team_avg_GA_last_3,away_team_avg_Sh_last_3
0,7,0,4,17,1.2,1.133333,0.777778,1.222222,12.777778,0.966667,...,15,1.5,1.822222,2.222222,0.888889,18.666667,1.866667,1.666667,0.333333,18.000000
1,6,2,2,10,1.3,1.822222,1.444444,1.000000,13.222222,1.633333,...,23,3.1,1.877778,1.555556,1.333333,14.888889,1.866667,1.666667,1.000000,17.333333
2,2,1,2,7,0.6,1.044444,0.666667,2.222222,11.222222,0.633333,...,6,1.1,0.811111,0.777778,2.555556,10.111111,0.933333,1.000000,2.666667,10.000000
3,0,2,2,13,1.0,1.766667,2.000000,0.888889,14.000000,1.600000,...,11,1.4,0.844444,0.777778,2.666667,9.111111,1.033333,0.666667,2.333333,10.666667
4,24,2,1,21,2.4,1.366667,1.222222,1.666667,12.000000,1.500000,...,10,2.4,2.188889,2.666667,1.000000,13.888889,2.533333,2.666667,0.666667,13.000000
5,23,1,4,14,1.1,1.488889,1.777778,1.777778,11.555556,1.466667,...,6,0.1,1.633333,1.000000,1.555556,15.444444,1.800000,1.333333,1.333333,18.000000
6,12,2,0,25,2.2,2.122222,2.222222,1.000000,16.777778,1.933333,...,19,2.9,1.188889,1.111111,1.333333,11.444444,1.466667,1.000000,1.000000,13.666667
7,4,1,2,5,0.8,1.933333,2.444444,2.000000,15.000000,1.600000,...,10,1.0,1.100000,0.888889,1.666667,10.666667,1.433333,1.000000,1.666667,13.333333
8,1,4,1,14,1.9,1.888889,2.555556,1.444444,14.888889,1.700000,...,13,1.7,1.244444,0.888889,1.888889,12.555556,1.266667,1.000000,1.666667,14.333333
9,15,2,1,14,1.1,1.533333,1.222222,1.444444,15.888889,1.266667,...,10,0.8,1.744444,2.111111,0.777778,15.444444,0.733333,1.000000,1.333333,12.333333


Encode the Home Team and Away Team, should be in the same way as in the training model

In [ ]:
#real_features = [[18,1.29,1.084,1.47,12.1,0.63333,0.33333,1.6667,8.3339,9,1.25,1.27,1.973,11.9,1.3,1.333,2.6667,9]]
#real_predictions = loaded_model.predict(real_features)

In [ ]:
realfeatures = np.array(df_gwfixfeatures)
real_predictions = loaded_model.predict(realfeatures)

In [ ]:
real_predictions

array([0, 2, 2, 2, 0, 0, 2, 2, 2, 0], dtype=int32)

In [ ]:
df_resultenc = pd.read_csv('{}/Encode tables/resultencoding.csv'.format(root))
df_resultenc

,Result,Result encoded
0,Away Win,0
1,Draw,1
2,Home Win,2


In [ ]:
df_realpred = pd.DataFrame({"Predicted Result": real_predictions})
df_realpred

,Predicted Result
0,0
1,2
2,2
3,2
4,0
5,0
6,2
7,2
8,2
9,0


In [ ]:
df_realpredenc = pd.merge(df_resultenc,df_realpred, left_on=['Result encoded'],right_on=['Predicted Result'], how='right')
df_realpredenc = df_realpredenc.drop(columns=['Result encoded','Predicted Result'])
df_realpredenc.rename(columns = {'Result':'Predicted Result'},inplace= True)
df_realpredenc

,Predicted Result
0,Away Win
1,Home Win
2,Home Win
3,Home Win
4,Away Win
5,Away Win
6,Home Win
7,Home Win
8,Home Win
9,Away Win


In [ ]:
df_gwpredictions =pd.concat([df_gwfix,df_realpredenc], axis =1)
df_gwpredictions

,Date,Home Team,Away Team,Predicted Result
0,27/10/2023,Crystal Palace,Tottenham,Away Win
1,28/10/2023,Chelsea,Brentford,Home Win
2,28/10/2023,Bournemouth,Burnley,Home Win
3,28/10/2023,Arsenal,Sheffield Utd,Home Win
4,28/10/2023,Wolves,Newcastle Utd,Away Win
5,29/10/2023,West Ham,Everton,Away Win
6,29/10/2023,Liverpool,Nott'ham Forest,Home Win
7,29/10/2023,Brighton,Fulham,Home Win
8,29/10/2023,Aston Villa,Luton Town,Home Win
9,29/10/2023,Manchester Utd,Manchester City,Away Win


In [ ]:
df_gwpredictions.to_csv('{}/Stored predictions/Predictions 2023 10 27.csv'.format(root), encoding='utf-8', index=False)

In [ ]:
df_gwpredictions.to_csv('{}/Stored prediction latest/Latest prediction.csv'.format(root), encoding='utf-8', index=False)

Do not run program further until all gameweek results have been revealed

In [ ]:
df_gwpredictions = pd.read_csv('{}/Stored predictions/Predictions 2023 10 20.csv'.format(root))

In [ ]:
df_gwresultinit = pd.read_csv('{}/2023 24 Results/Gameweek 9 results.csv'.format(root))
df_gwresultinit = df_gwresultinit.drop(columns=['Wk', 'Day','Time', 'xG', 'xG.1', 'Attendance','Venue','Referee','Match Report','Notes'])
print(df_gwresultinit.dtypes)
df_gwresultinit

Date     object
Home     object
Score    object
Away     object
dtype: object


,Date,Home,Score,Away
0,21/10/2023,Liverpool,2–0,Everton
1,21/10/2023,Bournemouth,1–2,Wolves
2,21/10/2023,Newcastle Utd,4–0,Crystal Palace
3,21/10/2023,Brentford,3–0,Burnley
4,21/10/2023,Manchester City,2–1,Brighton
5,21/10/2023,Nott'ham Forest,2–2,Luton Town
6,21/10/2023,Chelsea,2–2,Arsenal
7,21/10/2023,Sheffield Utd,1–2,Manchester Utd
8,22/10/2023,Aston Villa,4–1,West Ham
9,23/10/2023,Tottenham,2–0,Fulham


In [ ]:
df_gwresultinit[['homeGF','awayGF']] = df_gwresultinit['Score'].str.split('–',expand=True)
df_gwresultinit['Actual Result'] = df_gwresultinit.apply(result2, axis=1)
df_gwresult = df_gwresultinit.drop(columns=['Score','homeGF','awayGF'])
df_gwresult.rename(columns={'Home':'Home Team','Away':'Away Team'}, inplace=True)
df_gwresult

,Date,Home Team,Away Team,Actual Result
0,21/10/2023,Liverpool,Everton,Home Win
1,21/10/2023,Bournemouth,Wolves,Away Win
2,21/10/2023,Newcastle Utd,Crystal Palace,Home Win
3,21/10/2023,Brentford,Burnley,Home Win
4,21/10/2023,Manchester City,Brighton,Home Win
5,21/10/2023,Nott'ham Forest,Luton Town,Draw
6,21/10/2023,Chelsea,Arsenal,Draw
7,21/10/2023,Sheffield Utd,Manchester Utd,Away Win
8,22/10/2023,Aston Villa,West Ham,Home Win
9,23/10/2023,Tottenham,Fulham,Home Win


In [ ]:
df_predactualresult = pd.merge(df_gwpredictions,df_gwresult, on=['Date','Home Team','Away Team'])
df_predactualresult

,Date,Home Team,Away Team,Predicted Result,Actual Result
0,21/10/2023,Liverpool,Everton,Home Win,Home Win
1,21/10/2023,Bournemouth,Wolves,Home Win,Away Win
2,21/10/2023,Newcastle Utd,Crystal Palace,Home Win,Home Win
3,21/10/2023,Brentford,Burnley,Home Win,Home Win
4,21/10/2023,Manchester City,Brighton,Home Win,Home Win
5,21/10/2023,Nott'ham Forest,Luton Town,Draw,Draw
6,21/10/2023,Chelsea,Arsenal,Away Win,Draw
7,21/10/2023,Sheffield Utd,Manchester Utd,Away Win,Away Win
8,22/10/2023,Aston Villa,West Ham,Home Win,Home Win
9,23/10/2023,Tottenham,Fulham,Home Win,Home Win


In [ ]:
df_predactualresult.to_csv('{}/Stored predictions_results/Predictions results 2023 10 20.csv'.format(root), encoding='utf-8', index=False)

In [ ]:
#df_predactualresult = pd.read_csv('{}/Stored predictions_results/Predictions results 2023 08 25.csv'.format(root))

In [ ]:
df_predactual_master_init = pd.read_csv('{}/Stored predictions_results_master/Predictions results_master.csv'.format(root))

In [ ]:
df_predact_master = pd.concat([df_predactual_master_init, df_predactualresult], axis=0, ignore_index=True)
df_predact_master

#df_allteam = pd.concat(team_temp, axis=0, ignore_index=True)
#df_allteam

,Date,Home Team,Away Team,Predicted Result,Actual Result
0,11/08/2023,Burnley,Manchester City,Away Win,Away Win
1,12/08/2023,Arsenal,Nott'ham Forest,Home Win,Home Win
2,12/08/2023,Everton,Fulham,Home Win,Away Win
3,12/08/2023,Sheffield Utd,Crystal Palace,Away Win,Away Win
4,12/08/2023,Brighton,Luton Town,Home Win,Home Win
...,...,...,...,...,...
85,21/10/2023,Nott'ham Forest,Luton Town,Draw,Draw
86,21/10/2023,Chelsea,Arsenal,Away Win,Draw
87,21/10/2023,Sheffield Utd,Manchester Utd,Away Win,Away Win
88,22/10/2023,Aston Villa,West Ham,Home Win,Home Win


In [ ]:
df_predact_master.to_csv('{}/Stored predictions_results_master/Predictions results_master.csv'.format(root), encoding='utf-8', index=False)